# Creating a postgres database

For this project, I'll be creating a postgres database to store data on crimes that occurred in Boston from a csv file, and the database will contain the following:

- A table with appropriate datatypes for storing the data
- A schema 
- Readonly and readwrite groups with the appropriate privileges
- One user for each of these groups

### Creating a database and schema

Create a database named crime_db and a schema named crimes for storing the tables for containing the crime data.
The database crime_db does not exist yet so we connect to dq.

In [72]:
import psycopg2
import csv

In [73]:
conn = psycopg2.connect("dbname=dq user=dq")
# set autocommit to True bacause this is required for creating databases
conn.autocommit = True
cur = conn.cursor()
# create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()

ProgrammingError: database "crime_db" already exists


In [120]:
# connecting to the crime_db database
conn = psycopg2.connect("dbname=crime_db user=dq")
conn.autocommit = False
cur = conn.cursor()
# create the crimes schema
cur.execute("CREATE SCHEMA crimes;")

### Reading in the data

Reading in the csv file and getting the header row and the first row of data so that we can reference them when necessary.

In [121]:
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

### Preparing the data

Before we create a table to store the crime data, we need to identify the proper datatypes for the columns. I'll create a function that allows us to get a python set containing distinct values in each volumn which will help with:

- Checking whether an enumerated datatype might be a good choice for representing a column
- Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns

In [122]:
def get_col_set(csv_filename, col_index):
    values = set()
    with open(csv_filename,'r') as file:
        reader = csv.reader(file)
        # skipping the header row
        next(reader)
        for row in reader:
            values.add(row[col_index])
    return values

In [123]:
for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


Now that we know the number of distinct values in each column, we can explore converting some of the data types. As there are 2 text columns (description, day_of_the_week), we can convert the latter to enumerated data, while we'll check the max length of the former to decide what to do.

In [124]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [125]:
descriptions = get_col_set("boston.csv", 2)
max_len = 0
for d in descriptions:
    max_len = max(max_len, len(d))
print(max_len)

58


### Creating tables

In [126]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [127]:
ids = get_col_set("boston.csv", 1)
print(max(ids))

900


In [128]:
lons = get_col_set("boston.csv", 6)
max_len = 0
for l in lons:
    max_len = max(max_len, len(l))
print(max_len)
print(max(lons))

12
-71.17867378


#### Data types

Looking at the data in the first row, we can have the following data types:

- incident_number (max = 99999): integer
- offense_code (max = 900): smallint
- description (max length =58): varchar(100)
- date: date
- day_of_the_week: enumerated data
- lat (max length = 11, 2 digits before decimal, 8 after): decimal
- long (max length = 12, 2 digits and a negative before decimal, 8 after): decimal

First, we'll create and enumerated data type for day_of_the_week as there are only 7 distinct values for each day of the week.

In [129]:
days = get_col_set("boston.csv", 4)
print(days)

{'Wednesday', 'Saturday', 'Monday', 'Friday', 'Tuesday', 'Thursday', 'Sunday'}


In [130]:
# create the enumerated data atype
cur.execute("""
    CREATE TYPE enum_day AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code smallint,
        description VARCHAR(100),
        date DATE,
        day_of_the_week enum_day,
        lat decimal,
        long decimal
    );
""")


### Loading the data into the table

Now that we've set up the columns, we can load the data from the csv file. 

In [131]:
# Loading the data
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
    

In [132]:
# Checking that the data has been loaded properly
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

298329


### Creating appropriate user groups

I want to create two user groups: readonly and readwrite. The first step will be to revoke privileges from the public group and public schema.

In [133]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [134]:
# creating readonly user group
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")


In [135]:
# creating readwrite user group
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")


### Creating a user in each group

In [136]:
# Creating 2 different users
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist")

### Checking that privileges have been set up accordingly

Test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

- rolname: The name of the user / group that the privilege refers to.
- rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
- rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [137]:
# close the old connection to test with a brand new connection
conn.commit()
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()

In [141]:
# Checking roles
cur.execute("SELECT * FROM pg_roles")
for role in cur:
    print(role)
    
# getting specific columnes
# cur.execute("""
#     SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
#     WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
# """)

('dq', True, True, True, True, True, True, True, -1, '********', None, None, 10)
('readonly', False, True, False, False, False, False, False, -1, '********', None, None, 16527)
('readwrite', False, True, False, False, False, False, False, -1, '********', None, None, 16528)
('data_analyst', False, True, False, False, False, True, False, -1, '********', None, None, 16529)
('data_scientist', False, True, False, False, False, True, False, -1, '********', None, None, 16530)


In [139]:
# Checking for readonly group
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly';
    """)
print(cur.fetchall())

[('readonly', 'SELECT')]


In [140]:
# Checking for readwrite group
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';
    """)
print(cur.fetchall())

[('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]
